<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/03-03-visualize-net-income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DART 사이트에서 (삼성전자: 005930) 기업 공시 정보 가져오기 
1999-01-01부터 현재까지 분기/반기 보고서 가져와서 로컬 디렉토리에 csv로 저장하기

In [48]:
import requests
from io import BytesIO
import pandas as pd
import re

In [2]:
crtfc_key = ""
# with open('./sample_data/my_data.txt', 'r') as f:
#   crtfc_key = f.readline()
  # print(crtfc_key)

In [3]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [4]:
from datetime import datetime
now = datetime.today().strftime('%Y%m%d')

In [5]:
corp_code = '005930' # 삼성전자

In [6]:
def make_api_url(corp_code, bgn_de='19990101', end_de=now, pblntf_ty='A', pblntf_detail_ty=['A001','A002','A003'], page_count='100'):

  for i, code in enumerate(pblntf_detail_ty):
    pblntf_detail_ty[i] = 'pblntf_detail_ty=' + code
    if i != (len(pblntf_detail_ty)-1): pblntf_detail_ty[i] += '&'
  pblntf_detail_ty = "".join(pblntf_detail_ty)

  url = "https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={bgn_de}&end_de={end_de}&pblntf_ty={pblntf_ty}&{pblntf_detail_ty}&page_count={page_count}".format(\
                                                              crtfc_key=crtfc_key,corp_code=corp_code,bgn_de=bgn_de,end_de=end_de,pblntf_ty=pblntf_ty,pblntf_detail_ty=pblntf_detail_ty,page_count=page_count)
  return url

In [7]:
url = make_api_url(corp_code)
# print(url)

In [8]:
response = requests.get(url, headers={"user-agent":user_agent})

In [9]:
df_resp = pd.read_json(response.content)
df_resp

,status,message,page_no,page_count,total_count,total_page,list
0,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
1,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
2,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
3,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
4,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
...,...,...,...,...,...,...,...
88,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
89,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
90,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."
91,0,정상,1,100,93,1,"{'corp_code': '00126380', 'corp_name': '삼성전자',..."


In [10]:
list_resp = df_resp['list'].tolist()
df = pd.DataFrame.from_records(list_resp)
df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,분기보고서 (2021.03),20210517001185,삼성전자,20210517,
1,00126380,삼성전자,005930,Y,사업보고서 (2020.12),20210309000744,삼성전자,20210309,연
2,00126380,삼성전자,005930,Y,분기보고서 (2020.09),20201116001248,삼성전자,20201116,
3,00126380,삼성전자,005930,Y,반기보고서 (2020.06),20200814001766,삼성전자,20200814,
4,00126380,삼성전자,005930,Y,분기보고서 (2020.03),20200515001451,삼성전자,20200515,
...,...,...,...,...,...,...,...,...,...
88,00126380,삼성전자,005930,Y,분기보고서 (2000.03),20000515000739,삼성전자,20000515,
89,00126380,삼성전자,005930,Y,[기재정정]반기보고서 (1999.06),19990824000003,삼성전자,19990824,
90,00126380,삼성전자,005930,Y,반기보고서 (1999.06),19990814000182,삼성전자,19990814,정
91,00126380,삼성전자,005930,Y,[기재정정]사업보고서 (1998.12),19990626000007,삼성전자,19990626,연


In [11]:
report_nms = df['report_nm'].tolist()[:10]
rcept_nos = df['rcept_no'].tolist()[:10]

In [12]:
""" match text between two strings with regular-expression
 >>> s = 'Part 1. Part 2. Part 3 then more text'
 >>> re.search(r'Part 1\.(.*?)Part 3', s).group(1)
 ' Part 2. '
"""

dcm_nos = []
for rcept_no in rcept_nos:
  url =  'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}'.format(rcept_no=rcept_no)
  response = requests.get(url, headers={"user-agent":user_agent})
  dcm_nos.append(re.search(r"'{rcept_no}', '(.*?)',".format(rcept_no=rcept_no), response.content.decode('UTF-8')).group(1))

In [13]:
sheets = ["연결 재무상태표","연결 손익계산서","연결 포괄손익계산서","연결 자본변동표","연결 현금흐름표","재무상태표","손익계산서"]

def download_excel(report_nm, rcept_no, dcm_no):
    url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko".format(rcp_no=rcept_no,dcm_no=dcm_no)
    response = requests.get(url, headers={"user-agent":user_agent})
    f = BytesIO(response.content)

    for sheet in sheets:
        df = pd.read_excel(f, sheet_name=sheet, skiprows=5)
        df.to_csv(report_nm+"_"+sheet+".csv", encoding="euc-kr")

In [14]:
for report_nm, rcept_no, dcm_no in zip(report_nms, rcept_nos, dcm_nos):
  download_excel(report_nm, rcept_no, dcm_no)

In [15]:
df = pd.read_csv('분기보고서 (2021.03)_연결 포괄손익계산서.csv',encoding="euc-kr")

In [16]:
df

,Unnamed: 0,,제 53 기 1분기,Unnamed: 2,제 52 기 1분기,Unnamed: 4
0,0,NaN,3개월,누적,3개월,누적
1,1,당기순이익(손실),7141685,7141685,4884926,4884926
2,2,기타포괄손익,4299126,4299126,1039168,1039168
3,3,후속적으로 당기손익으로 재분류되지 않는 포괄손익,1866697,1866697,-1189281,-1189281
4,4,기타포괄손익-공정가치금융자산평가손익,1885011,1885011,-1109611,-1109611
5,5,관계기업 및 공동기업의 기타포괄손익에 대한 지분,14425,14425,-46974,-46974
6,6,순확정급여부채(자산) 재측정요소,-32739,-32739,-32696,-32696
7,7,후속적으로 당기손익으로 재분류되는 포괄손익,2432429,2432429,2228449,2228449
8,8,관계기업 및 공동기업의 기타포괄손익에 대한 지분,-53921,-53921,28944,28944
9,9,해외사업장환산외환차이,2457265,2457265,2228543,2228543


In [17]:
net_income = df.iloc[1][2]

In [68]:
import glob

fileList = glob.glob('*_연결 포괄손익계산서.csv')
fileList

['분기보고서 (2021.03)_연결 포괄손익계산서.csv',
 '분기보고서 (2019.03)_연결 포괄손익계산서.csv',
 '반기보고서 (2020.06)_연결 포괄손익계산서.csv',
 '사업보고서 (2018.12)_연결 포괄손익계산서.csv',
 '사업보고서 (2019.12)_연결 포괄손익계산서.csv',
 '분기보고서 (2020.09)_연결 포괄손익계산서.csv',
 '분기보고서 (2019.09)_연결 포괄손익계산서.csv',
 '반기보고서 (2019.06)_연결 포괄손익계산서.csv',
 '사업보고서 (2020.12)_연결 포괄손익계산서.csv',
 '분기보고서 (2020.03)_연결 포괄손익계산서.csv']

In [72]:
df.iloc[1][2]

'7141685'

In [78]:
d = {}

for file in fileList:
  df = pd.read_csv(file,encoding="euc-kr")
  net_income = df.iloc[1][2]
  d[file[7:14]] = int(net_income)
  # print('filename:', file[7:14], 'net income:', net_income)

In [79]:
d

{'2018.12': 44344857,
 '2019.03': 5043585,
 '2019.06': 5180576,
 '2019.09': 6287664,
 '2019.12': 21738865,
 '2020.03': 4884926,
 '2020.06': 5555083,
 '2020.09': 9360693,
 '2020.12': 26407832,
 '2021.03': 7141685}